In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
import copy

In [2]:
class OrderParameter:

    # name should be unique to the Order Parameter being defined
    # In other words for every possible pair of OP's x and y, (x.name != y.name) must be true
    def __init__(self, name, traj):
        self.name = name
        self.traj = traj
        
    def __eq__(self, other):
        return self.name == other.name
    
    def __hash__(self):
        return hash(self.name)
        
    def __str__(self):
        return str(self.name)

In [3]:
def distortion(centers, ops, mut):
    dis = 0.0
    for i in ops:
        min_val = np.inf
        for j in centers:
            tmp = mut.iqr(i, j)
            if tmp < min_val:
                min_val = tmp
        dis = dis + (min_val * min_val)
    return 1 + (dis ** (0.5))

In [4]:
class DissimilarityMatrix:

    def __init__(self, max_OPs, mut):
        self.max_OPs = max_OPs
        self.matrix = [[] for i in range(max_OPs)]
        self.mut = mut
        self.OPs = []

    def add_OP(self, OP):
        if len(self.OPs) == self.max_OPs:
            mut_info = []
            existing = []
            for i in range(len(self.OPs)):
                mut_info.append(self.mut.iqr(self.OPs[i], OP))
                product = 1
                for j in range(len(self.OPs)):
                    if not i == j:
                        product = product * self.matrix[i][j]
                existing.append(product)
            update = False
            difference = None
            for i in range(len(self.OPs)):
                candidate_info = 1
                for j in range(len(self.OPs)):
                    if not i == j:
                        candidate_info = candidate_info * mut_info[j]
                if candidate_info > existing[i]:
                    update = True
                    if difference == None:
                        difference = candidate_info - existing[i]
                        old_OP = i
                    else:
                        if (candidate_info - existing[i]) > difference:
                            difference = candidate_info - existing[i]
                            old_OP = i
            if update == True:
                mut_info[old_OP] = self.mut.iqr(OP, OP)
                self.matrix[old_OP] = mut_info
                self.OPs[old_OP] = OP
                for i in range(len(self.OPs)):
                    self.matrix[i][old_OP] = mut_info[i]
        else:
            for i in range(len(self.OPs)):
                mut_info = self.mut.iqr(OP, self.OPs[i])
                self.matrix[i].append(mut_info)
                self.matrix[len(self.OPs)].append(mut_info)
            self.matrix[len(self.OPs)].append(self.mut.iqr(OP, OP))
            self.OPs.append(OP)
        
    def reduce(self):
        min_val = 10
        index = -1
        for i in range(len(self.matrix)):
            product = 1
            for j in range(len(self.matrix[i])):
                if not i == j:
                    product = product * self.matrix[i][j]
            if product < min_val:
                index = i
                min_val = product
        self.matrix.pop(index)
        for i in range(len(self.matrix)):
            self.matrix[i].pop(index)
        self.OPs.pop(index)
        
    def min_product(self):
        min_val = 10
        for i in range(len(self.matrix)):
            product = 1
            for j in range(len(self.matrix[i])):
                if not i == j:
                    product = product * self.matrix[i][j]
            if product < min_val:
                min_val = product
        return min_val
    
    def get_OPs(self):
        return self.OPs
    
    def __str__(self):
        output = ""
        output = output + "OPs:\n"
        for i in self.OPs:
            output = output + str(i) + "\n"
        output = output + "\nMatrix:\n"
        for i in self.matrix:
            for j in i:
                output = output + str(j) + " "
            output = output + "\n"
        return output
                

In [5]:
def d1_bin(x, bins = 80):

    min_val = np.amin(x)
    max_val = np.amax(x)
    span = max_val - min_val

    p_x = [0.0 for i in range(bins)]

    for i in x:
        bin_num = (int) (bins * (i - min_val) / span)
        if bin_num == bins:
            bin_num -= 1
        p_x[bin_num] += 1.0 / len(x)

    return p_x

In [6]:
def d2_bin(x, y, bins = 80):

    if len(x) != len(y):
        raise Exception("Order parameter lists are of different size.")

    min_x = np.amin(x)
    max_x = np.amax(x)
    span_x = max_x - min_x

    min_y = np.amin(y)
    max_y = np.amax(y)
    span_y = max_y - min_y

    p_xy = [[0.0 for i in range(bins)] for j in range(bins)]

    for i in range(len(x)):
        bin_x = (int) (bins * (x[i] - min_x) / span_x)
        bin_y = (int) (bins * (y[i] - min_y) / span_y)
        if bin_x == bins:
            bin_x -= 1
        if bin_y == bins:
            bin_y -= 1
        p_xy[bin_x][bin_y] += 1.0 / len(x)

    return p_xy

In [7]:
class Memoizer:
    
    def __init__(self):
        self.memo = {}
        self.bins = 80
        
    def iqr(self, OP1, OP2):
        index = str(OP1.name) + " " + str(OP2.name)
        if index in self.memo:
            return self.memo[index]
        else:
            x = OP1.traj
            y = OP2.traj
            p_x = d1_bin(x, self.bins)
            p_y = d1_bin(y, self.bins)
            p_xy = d2_bin(x, y, self.bins)

            info = 0
            entropy = 0

            for i in range(len(p_x)):
                for j in range(len(p_y)):
                    if p_xy[i][j] != 0:
                        entropy -= p_xy[i][j] * np.log(p_xy[i][j])
                        info += p_xy[i][j] * np.log(p_xy[i][j] / (p_x[i] * p_y[j]))

            if ((1 - (info / entropy)) < 0):
                output = 0.0
            else:
                output = (1 - (info / entropy))
            self.memo[index] = output
            return output
    
    def __str__(self):
        print(len(self.memo))

In [8]:
def grouping(new_OPs, all_OPs, mut):
    groups = [[] for i in range(len(new_OPs))]
    for OP in all_OPs:
        group = 0
        for i in range(len(new_OPs)):
            tmp = mut.iqr(OP, new_OPs[i])
            if tmp < mut.iqr(OP, new_OPs[group]):
                group = i
        groups[group].append(OP)
    return groups

In [9]:
def group_evaluation(OPs, mut):
    
    center = OPs[0]
    min_distortion = distortion([OPs[0]], OPs, mut)
    
    for i in OPs:
        tmp = distortion([i], OPs, mut)
        if tmp < min_distortion:
            center = i
            min_distortion = tmp

    return center

In [10]:
def cluster(ops, seeds, mut):
    
    old_centers = []
    centers = copy.deepcopy(seeds)
    
    while (set(centers) != set(old_centers)):
        
        old_centers = copy.deepcopy(centers)
        centers = []
        groups = grouping(old_centers, ops, mut)
        
        for i in range(len(groups)):
            result = group_evaluation(groups[i], mut)
            centers.append(result)

    return centers


In [11]:
def fix_distortion(lst):
    for i in range(len(lst) - 1):
        index = len(lst) - i - 1
        if lst[index] < lst[index - 1]:
            lst[index - 1] = lst[index]

In [12]:
def find_ops(old_ops, max_outputs):
    
    mut = Memoizer()
    matrix = DissimilarityMatrix(max_outputs, mut)
    
    for i in old_ops:
        matrix.add_OP(i)
        
    for i in old_ops[::-1]:
        matrix.add_OP(i)
    
    tmp = copy.deepcopy(matrix)
    distortion_array = []
    num_array = []

    while (len(tmp.OPs) > 0):
        num_array.append(len(tmp.OPs))
        seed = []
        for i in tmp.OPs:
            seed.append(i)
        tmp_ops = cluster(old_ops, seed, mut)
        distortion_array.append(distortion(tmp_ops, old_ops, mut))
        tmp.reduce()
    
    fix_distortion(distortion_array)
    
    jumps = []
    
    for dim in range(1,7):
        neg_expo = np.array(distortion_array) ** (-0.5 * dim)
        local = []
        for j in range(len(neg_expo) - 1):
            local.append(neg_expo[j] - neg_expo[j + 1])
        jumps.append(local)
    
    num_ops = 0
    
    for dim in range(1,7):
        neg_expo = np.array(distortion_array) ** (-0.5 * dim)
        jumps = []
        for i in range(len(neg_expo) - 1):
            jumps.append(neg_expo[i] - neg_expo[i + 1])
            
        min_index = 0
        for i in range(len(jumps)):
            if jumps[i] > jumps[min_index]:
                min_index = i
        if num_array[min_index] > num_ops:
            num_ops = num_array[min_index]
        
    while (len(matrix.OPs) > num_ops):
        matrix.reduce()
        
    centers = cluster(old_ops, matrix.OPs, mut)
    return centers


In [13]:
def gen_two():
    traj = []
    for i in range(5000):
        tmp = random.random()
        tmp = 1 - (tmp * tmp * tmp * tmp)
        if random.random() < 0.5:
            tmp = -1 * tmp
        traj.append(tmp)
    return traj

In [14]:
def gen_three():
    traj = []
    for i in range(5000):
        tmp = random.random()
        tmp = (tmp * tmp * tmp * tmp)/2
        if random.random() < 0.5:
            tmp = 1 - tmp
        if random.random() < 0.5:
            tmp = -1 * tmp
        traj.append(tmp)
    return traj

In [15]:
def add_noise(traj):
    new_traj = []
    for i in traj:
        new_traj.append(i + (random.random() / 10) - 0.05)
    return new_traj

In [16]:
def rand_insert(lst, ele):
    lst.insert(random.randrange(len(lst) + 1), ele)

In [17]:
def make_OP(name, num, num_wells, ops):
    
    original = None
    if num_wells == 2:
        original = gen_two()
    elif num_wells == 3:
        original = gen_three()
    else:
        raise Exception("I've only built in distribution generators for 2 and 3 well OP's.")
        
    rand_insert(ops, OrderParameter(name, original))
    
    for i in range(num - 1):
        rand_insert(ops, OrderParameter(name + str(i + 1), add_noise(original)))
        
    return None
        

In [18]:
ten_ops = []
make_OP("A", 14, 2, ten_ops)
make_OP("B", 10, 2, ten_ops)
make_OP("C", 14, 3, ten_ops)
make_OP("D", 10, 3, ten_ops)
make_OP("E", 14, 2, ten_ops)
make_OP("F", 10, 2, ten_ops)
make_OP("G", 14, 2, ten_ops)
make_OP("H", 10, 3, ten_ops)
make_OP("I", 14, 3, ten_ops)
make_OP("J", 10, 2, ten_ops)

In [19]:
final_ten_ops = find_ops(ten_ops, 30)

In [20]:
print("Number of Order Parameters: " + str(len(final_ten_ops)))
for i in final_ten_ops:
    print(i.name)
print("\nAll order Parmeters:")
for i in ten_ops:
    print(i.name)

Number of Order Parameters: 10
A
H
F
I
G
D
C
E
B
J

All order Parmeters:
H5
D7
I4
I2
C12
A12
F7
A2
A9
F1
I9
F5
G6
F3
B1
I6
A10
D
E13
G2
E11
B6
G10
A5
J1
C1
G8
J6
B9
G9
E9
H4
G12
A1
E5
F9
I
I5
E7
H7
B8
F2
B2
F8
A11
I8
C8
I12
A7
J5
C
E6
E
B4
H8
H3
H1
G1
I7
A4
J
J7
E10
I3
E3
D1
G4
J8
G7
H
C6
E12
C11
D4
G3
E1
J4
B
G11
I10
B3
G
E2
C7
D8
C4
A8
C2
B5
A3
H9
G5
B7
H2
E8
J2
I11
J3
F6
A6
F
I13
D9
A13
G13
D6
D3
C5
I1
D2
C10
J9
E4
F4
A
H6
D5
C13
C3
C9


In [21]:
five_ops = []
make_OP("A", 14, 2, five_ops)
make_OP("B", 10, 2, five_ops)
make_OP("C", 14, 3, five_ops)
make_OP("D", 10, 3, five_ops)
make_OP("E", 14, 2, five_ops)

In [22]:
final_five_ops = find_ops(five_ops, 30)

In [23]:
print("Number of Order Parameters: " + str(len(final_five_ops)))
for i in final_five_ops:
    print(i.name)
print("\nAll order Parmeters:")
for i in five_ops:
    print(i.name)

Number of Order Parameters: 5
E
C
B
D
A

All order Parmeters:
B
E12
E13
E4
E9
A
C1
D7
B5
C13
A3
D3
C6
C4
E5
B2
D6
C5
E2
A2
C7
A8
C10
C2
A11
A4
A10
E3
D1
B4
C8
D9
E7
C
A12
D2
E8
C3
B3
B7
D4
A13
D8
A6
A7
A9
A1
B8
B9
E1
A5
E10
E6
C9
C12
B6
D5
E11
E
D
B1
C11
